## Calculate surface-forced watermass transformation
Presently, information about the products, e.g. names of variables is hard-coded. Could improve robustness by specifying generic names.

In [20]:
%load_ext autoreload
%autoreload 2
from dask.diagnostics import ProgressBar
import so_decadal_variability.process as process
import so_decadal_variability.grid as sogrid
import so_decadal_variability.wmt as wmt
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
fluxname='era5'
oceanname='iap'
timeslice = slice('1980','2018')
dg = 0.1
gn_edges = np.arange(1026,1028.5,dg)
save = False
ds = process.loaddata(fluxname,oceanname,timeslice)
ds, xgrid = sogrid.get_xgcm(ds)
G = wmt.calc_watermasstransformation(ds,xgrid,gn_edges)
if save:
    with ProgressBar():
        G.to_netcdf('../data/processed/'+fluxname+'_'+oceanname+'/G_'+str(dg)+'.nc')

KeyError: 'b'

In [19]:
ds

<xarray.Dataset>
Dimensions:    (depth: 41, lat: 60, lon: 360, time: 468)
Coordinates:
  * time       (time) datetime64[ns] 1980-01-01 1980-02-01 ... 2018-12-01
  * lon        (lon) int64 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
  * lat        (lat) float32 -89.5 -88.5 -87.5 -86.5 ... -33.5 -32.5 -31.5 -30.5
  * depth      (depth) float32 1.0 5.0 10.0 20.0 ... 1600.0 1700.0 1800.0 2000.0
Data variables:
    fw         (time, lat, lon) float32 dask.array<chunksize=(468, 60, 360), meta=np.ndarray>
    ht         (time, lat, lon) float32 dask.array<chunksize=(468, 60, 360), meta=np.ndarray>
    sr         (time, lat, lon) float32 dask.array<chunksize=(468, 60, 360), meta=np.ndarray>
    alpha      (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    beta       (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    ct         (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    gamman     (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    pt0        (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    rho        (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    s          (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    sa         (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    sigma0     (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    sigma1     (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    sigma2     (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    sigma3     (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    sigma4     (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    t          (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    cell_area  (lat, lon) float64 dask.array<chunksize=(60, 360), meta=np.ndarray>
    dpth4d     (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    dx         (lat, lon) float32 dask.array<chunksize=(60, 360), meta=np.ndarray>
    dy         (lat, lon) float32 dask.array<chunksize=(60, 360), meta=np.ndarray>
    dz4d       (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    lat4d      (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    lon4d      (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    mask       (lat, lon) float64 dask.array<chunksize=(60, 360), meta=np.ndarray>
    prs4d      (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>
    vol4d      (time, depth, lat, lon) float32 dask.array<chunksize=(468, 41, 60, 360), meta=np.ndarray>

In [8]:
oceands

<xarray.Dataset>
Dimensions:    (depth_std: 41, lat: 60, lon: 360, time: 480)
Coordinates:
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2018-12-01
  * lon        (lon) float32 -180.0 -179.0 -178.0 -177.0 ... 177.0 178.0 179.0
  * lat        (lat) float32 -89.5 -88.5 -87.5 -86.5 ... -33.5 -32.5 -31.5 -30.5
  * depth_std  (depth_std) float32 1.0 5.0 10.0 20.0 ... 1700.0 1800.0 2000.0
Data variables:
    alpha      (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    beta       (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    ct         (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    gamman     (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    pt0        (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    rho        (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    s          (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    sa         (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    sigma0     (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    sigma1     (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    sigma2     (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    sigma3     (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    sigma4     (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
    t          (time, depth_std, lat, lon) float32 dask.array<chunksize=(480, 41, 60, 360), meta=np.ndarray>
Attributes:
    CDI:                       Climate Data Interface version 1.9.3 (http://m...
    history:                   Wed Nov 04 08:59:12 2020: cdo setname,alpha /r...
    Conventions:               CF-1.6
    Title:                     IAP 3-Dimentional Subsurface Temperature Datas...
    StartYear:                 1979
    StartMonth:                1
    StartDay:                  1
    EndYear:                   1979
    EndMonth:                  1
    EndDay:                    30
    Period:                    1
    GridProjection:            Mercator, gridded
    GridPoints:                360x180
    Creator:                   Lijing Cheng From IAP,CAS,P.R.China
    Reference:                 Cheng and Zhu 2016. Journal of Climate; Cheng ...
    NCO:                       netCDF Operators version 4.7.5 (Homepage = htt...
    nco_openmp_thread_number:  1
    CDO:                       Climate Data Operators version 1.9.3 (http://m...